In [1]:
import numpy as np
import pandas as pd
from deepchem.utils.save import log

/home/sohompaul/psi4conda/envs/deepchem/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
RDKit WARNING: [08:52:05] Enabling RDKit 2019.09.3 jupyter extensions
/home/sohompaul/psi4conda/envs/deepchem/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sohompaul/psi4conda/envs/deepchem/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (typ

In [5]:
LOAD_FILE_NAME = 'az_ppb.csv'
# WRITE_FILE_NAME = 'az_ppb_last_col_dropped.csv'
WRITE_FILE_NAME = 'az_ppb_clean.csv'

In [6]:
data = pd.read_csv(LOAD_FILE_NAME)

In [7]:
data.head()

,canonical_smiles,molregno,standard_relation,standard_value,standard_units
0,CC(C)(C(=O)c1cccnc1)c2cccnc2,86284,NaN,11.18,%
1,CN(CC(=O)O)NC(=O)C[C@H](N)C[C@@H](O)CN,127808,NaN,12.63,%
2,OC(Cn1cncn1)(Cn2cncn2)c3ccc(F)cc3F,13758,NaN,14.23,%
3,CCN(CC)CCNC(=O)c1ccc(NS(=O)(=O)C)cc1,160578,NaN,15.10,%
4,COc1ccc(cc1)C(CN(C)C)C2(O)CCCCC2,27278,NaN,18.64,%


In [8]:
print(data.columns)

Index(['canonical_smiles', 'molregno', 'standard_relation', 'standard_value',
       'standard_units'],
      dtype='object')


In [14]:
data = data.drop(columns=[data.columns[1], data.columns[2], data.columns[-1]])

In [15]:
data.to_csv(WRITE_FILE_NAME)

In [ ]:
def split(dataset,
          frac_train=.80,
          frac_valid=.10,
          frac_test=.10,
          log_every_n=1000):
    """
    Splits internal compounds into train/validation/test by scaffold.
    """
    np.testing.assert_almost_equal(frac_train + frac_valid + frac_test, 1.)
    scaffolds = {}
    log("About to generate scaffolds", True)
    data_len = len(dataset)

    for ind, smiles in enumerate(dataset):
        if ind % log_every_n == 0:
            log("Generating scaffold %d/%d" % (ind, data_len), True)
        scaffold = generate_scaffold(smiles)
        if scaffold not in scaffolds:
            scaffolds[scaffold] = [ind]
        else:
            scaffolds[scaffold].append(ind)

    scaffolds = {key: sorted(value) for key, value in scaffolds.items()}
    scaffold_sets = [
        scaffold_set
        for (scaffold, scaffold_set) in sorted(
            scaffolds.items(), key=lambda x: (len(x[1]), x[1][0]), reverse=True)
    ]
    train_cutoff = frac_train * len(dataset)
    valid_cutoff = (frac_train + frac_valid) * len(dataset)
    train_inds, valid_inds, test_inds = [], [], []
    log("About to sort in scaffold sets", True)
    for scaffold_set in scaffold_sets:
        if len(train_inds) + len(scaffold_set) > train_cutoff:
            if len(train_inds) + len(valid_inds) + len(scaffold_set) > valid_cutoff:
                test_inds += scaffold_set    
            else:
                valid_inds += scaffold_set
        else:
            train_inds += scaffold_set
    return train_inds, valid_inds, test_inds

In [ ]:
def donkey_load_dataset(filename, whiten=False):
    f = open(filename, 'r')
    features = []
    labels = []
    tracer = 0
    for line in f:
        if tracer == 0:
            tracer += 1
            continue
        splits =  line[:-1].split(',')
        features.append(splits[-1])
        labels.append(float(splits[-2]))
    features = np.array(features)
    labels = np.array(labels, dtype='float32').reshape(-1, 1)

    train_ind, val_ind, test_ins = split(features)

    train_features = np.take(features, train_ind)
    train_labels = np.take(labels, train_ind)
    val_features = np.take(features, val_ind)
    val_labels = np.take(labels, val_ind)

    return train_features, train_labels, val_features, val_labels

In [ ]:
donkey_load_dataset(WRITE_FILE_NAME)